In [ ]:
import glob
import os
import gc
import pandas as pd
import geopy.distance
pd.set_option('display.max_columns', None)  
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def reduce_memory(df):
    for c in df.columns:
        if df[c].dtype=='int':
            if df[c].min()<0:
                if df[c].abs().max()<2**7:
                    df[c] = df[c].astype('int8')
                elif df[c].abs().max()<2**15:
                    df[c] = df[c].astype('int16')
                elif df[c].abs().max()<2**31:
                    df[c] = df[c].astype('int32')
                else:
                    continue
            else:
                if df[c].max()<2**8:
                    df[c] = df[c].astype('uint8')
                elif df[c].max()<2**16:
                    df[c] = df[c].astype('uint16')
                elif df[c].max()<2**32:
                    df[c] = df[c].astype('uint32')
                else:
                    continue
    return df

In [2]:
# A loop to display each dataset's schema 
for files in glob.glob("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\*.csv"):
    df = pd.read_csv(files)
    print (files, 'has ', len(df.columns), 'columns\n')
    print (df.columns, '\n')
    print (df.dtypes, '\n')

D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2011(geocoded).csv has  13 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'full_address',
       'Latitude', 'Longitude', 'storey_range', 'floor_area_sqm', 'flat_model',
       'lease_commence_date', 'resale_price'],
      dtype='object') 

month                   object
town                    object
flat_type               object
block                   object
street_name             object
full_address            object
Latitude               float64
Longitude              float64
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
dtype: object 

D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2012 Jan Feb.csv has  13 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'full_address',
       'Latitude', 'Longitude', 'storey_range', 'floor_area_sqm', 

In [3]:
flat_coord = pd.DataFrame()
for files in glob.glob("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\*.csv"):
    print ('Concatenating...', files)
    flat_coord = pd.concat([flat_coord,pd.read_csv(files).iloc[:,:]],axis=0)
print ('Datasets are now concatenated into one dataframe.')


Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2011(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2012 Jan Feb.csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2012-2014(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2015-2016(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2017(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2018(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2019(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2020(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2021(geocoded).csv
Datasets are now concatenated into one

In [4]:
flat_coord.shape


(183905, 14)

In [5]:
flat_coord=flat_coord[["full_address","Latitude","Longitude","month","town","flat_type","block","street_name","storey_range","floor_area_sqm","flat_model","lease_commence_date","resale_price","remaining_lease"]]

In [7]:
supermarket_coord = pd.read_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\supermarket_coordinates_clean.csv")
supermarket_coord = supermarket_coord[['address','LATITUDE','LONGITUDE']]
supermarket_coord.head()

,address,LATITUDE,LONGITUDE
0,GOLDEN VILLAGE (GV KATONG),1.305225,103.905102
1,MULTI STOREY CAR PARK,1.273388,103.808981
2,11 RIVERVALE CRESCENT SINGAPORE 545082,1.392504,103.904484
3,CORALINUS,1.403719,103.906138
4,WESTLITE JUNIPER,1.406303,103.760615


In [8]:
from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [9]:
nearest_supermarket = find_nearest(flat_coord, supermarket_coord)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T
flat_supermarket = flat_supermarket.rename(columns={0: 'full_address', 1: 'supermarket', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop(['index','supermarket'], axis=1)
flat_supermarket.head()

,full_address,supermarket_dist,num_supermarket_2km
0,"45 TELOK BLANGAH DR,Singapore",0.155644,11
1,"106 BT PURMEI RD,Singapore",0.875532,16
2,"50 TELOK BLANGAH DR,Singapore",0.126816,11
3,"108 BT PURMEI RD,Singapore",0.835485,15
4,"110 SPOTTISWOODE PK RD,Singapore",0.784717,17


In [12]:
flat_supermarket.to_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\flat_supermarket.csv")

In [13]:
flat_supermarket

,full_address,supermarket_dist,num_supermarket_2km
0,"45 TELOK BLANGAH DR,Singapore",0.155644,11
1,"106 BT PURMEI RD,Singapore",0.875532,16
2,"50 TELOK BLANGAH DR,Singapore",0.126816,11
3,"108 BT PURMEI RD,Singapore",0.835485,15
4,"110 SPOTTISWOODE PK RD,Singapore",0.784717,17
...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",0.129043,15
16135,"512A YISHUN ST 51,Singapore",0.686393,12
16136,"506A YISHUN AVE 4,Singapore",0.547579,13
16137,"83 COMMMONWEALTH CLOSE,Singapore",0.413755,14


In [74]:
merged_flat_hawker=pd.read_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\merged_flat_mrt_school_mall_hawker.csv")
merged_flat_hawker=merged_flat_hawker.drop(["Unnamed: 0"],axis=1)
merged_flat_hawker

C:\Users\65983\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Blk 112 Jalan Bukit Merah,0.767276,6
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blk 11 Telok Blangah Crescent,1.063867,5
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Blk 112 Jalan Bukit Merah,0.728191,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Chong Pang Market & Food Centre,3.117045,0
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Chong Pang Market & Food Centre,3.117045,0
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,Commonwealth Crescent Market,0.299702,4
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,Commonwealth Crescent Market,0.243151,4


In [75]:
flat_mall=pd.read_csv('D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\merged_flat_mrt_school_mal.csv')

In [76]:
flat_mall=flat_mall.drop(["Unnamed: 0"],axis=1)

In [77]:
flat_mall

,full_address,mall,mall_dist,num_mall_2km
0,"45 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.454797,5
1,"45 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.454797,5
2,"106 BT PURMEI RD,Singapore",Radin Mas Primary School,0.265618,7
3,"50 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.372768,5
4,"108 BT PURMEI RD,Singapore",Radin Mas Primary School,0.221545,8
...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",Northoaks Primary School,0.073958,6
183901,"463B SEMBAWANG DR,Singapore",Northoaks Primary School,0.073958,6
183902,"82 COMMMONWEALTH CLOSE,Singapore",New Town Primary School,0.498291,4
183903,"83 COMMMONWEALTH CLOSE,Singapore",New Town Primary School,0.554578,5


In [78]:
flat_hawker_mall=merged_flat_hawker.join(flat_mall[["mall","mall_dist","num_mall_2km"]])
flat_hawker_mall

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km,mall,mall_dist,num_mall_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Blk 112 Jalan Bukit Merah,0.767276,6,Radin Mas Primary School,0.265618,7
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blk 11 Telok Blangah Crescent,1.063867,5,Blangah Rise Primary School,0.372768,5
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Blk 112 Jalan Bukit Merah,0.728191,6,Radin Mas Primary School,0.221545,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,Commonwealth Crescent Market,0.299702,4,New Town Primary School,0.498291,4
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,Commonwealth Crescent Market,0.243151,4,New Town Primary School,0.554578,5


In [79]:
flat_mrt=pd.read_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\flat_mrt.csv")

In [80]:
flat_mrt=flat_mrt.rename(columns={"flat":"full_address"})
flat_mrt

,Unnamed: 0,full_address,mrt,mrt_dist,num_mrt_2km
0,0,"45 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.153912,4
1,1,"106 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.962478,6
2,2,"50 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.260606,5
3,3,"108 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.988774,6
4,4,"110 SPOTTISWOODE PK RD,Singapore",OUTRAM PARK MRT STATION,0.759241,11
...,...,...,...,...,...
16134,16134,"316A PUNGGOL WAY,Singapore",SUMANG LRT STATION,0.054245,18
16135,16135,"512A YISHUN ST 51,Singapore",KHATIB MRT STATION,0.991286,2
16136,16136,"506A YISHUN AVE 4,Singapore",KHATIB MRT STATION,0.815135,2
16137,16137,"83 COMMMONWEALTH CLOSE,Singapore",COMMONWEALTH MRT STATION,0.355395,7


In [82]:
flat_mrt=flat_mrt.drop(["Unnamed: 0"],axis=1)

In [83]:
flat_mrt

,full_address,mrt,mrt_dist,num_mrt_2km
0,"45 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.153912,4
1,"106 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.962478,6
2,"50 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.260606,5
3,"108 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.988774,6
4,"110 SPOTTISWOODE PK RD,Singapore",OUTRAM PARK MRT STATION,0.759241,11
...,...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",SUMANG LRT STATION,0.054245,18
16135,"512A YISHUN ST 51,Singapore",KHATIB MRT STATION,0.991286,2
16136,"506A YISHUN AVE 4,Singapore",KHATIB MRT STATION,0.815135,2
16137,"83 COMMMONWEALTH CLOSE,Singapore",COMMONWEALTH MRT STATION,0.355395,7


In [85]:
flat_mrt=pd.merge(flat_coord, flat_mrt, on='full_address', how='left')

In [86]:
flat_hawker_mall_mrt=flat_hawker_mall.join(flat_mrt[["mrt","mrt_dist","num_mrt_2km"]])
flat_hawker_mall_mrt

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Blk 112 Jalan Bukit Merah,0.767276,6,Radin Mas Primary School,0.265618,7,HARBOURFRONT MRT STATION,0.962478,6
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blk 11 Telok Blangah Crescent,1.063867,5,Blangah Rise Primary School,0.372768,5,TELOK BLANGAH MRT STATION,0.260606,5
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Blk 112 Jalan Bukit Merah,0.728191,6,Radin Mas Primary School,0.221545,8,HARBOURFRONT MRT STATION,0.988774,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,Commonwealth Crescent Market,0.299702,4,New Town Primary School,0.498291,4,COMMONWEALTH MRT STATION,0.316139,7
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,Commonwealth Crescent Market,0.243151,4,New Town Primary School,0.554578,5,COMMONWEALTH MRT STATION,0.355395,7


In [87]:

flat_school=pd.read_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\flat_school.csv")
flat_school=flat_school.drop(["Unnamed: 0"],axis=1)
flat_school

,full_address,school,school_dist,num_school_2km
0,"45 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.454797,5
1,"106 BT PURMEI RD,Singapore",Radin Mas Primary School,0.265618,7
2,"50 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.372768,5
3,"108 BT PURMEI RD,Singapore",Radin Mas Primary School,0.221545,8
4,"110 SPOTTISWOODE PK RD,Singapore",Cantonment Primary School,0.438143,5
...,...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",Valour Primary School,0.232633,11
16135,"512A YISHUN ST 51,Singapore",Naval Base Primary School,0.360785,6
16136,"506A YISHUN AVE 4,Singapore",Naval Base Primary School,0.156402,7
16137,"83 COMMMONWEALTH CLOSE,Singapore",New Town Primary School,0.554578,5


In [88]:
flat_school=pd.merge(flat_coord, flat_school, on='full_address', how='left')

In [89]:
flat_school

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,school,school_dist,num_school_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blangah Rise Primary School,0.454797,5
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blangah Rise Primary School,0.454797,5
2,"106 BT PURMEI RD,Singapore",1.273075,103.825657,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Radin Mas Primary School,0.265618,7
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blangah Rise Primary School,0.372768,5
4,"108 BT PURMEI RD,Singapore",1.273442,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Radin Mas Primary School,0.221545,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Northoaks Primary School,0.073958,6
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Northoaks Primary School,0.073958,6
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,New Town Primary School,0.498291,4
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,New Town Primary School,0.554578,5


In [90]:
flat_hawker_mall_mrt_school=flat_hawker_mall_mrt.join(flat_school[["school","school_dist","num_school_2km"]])
flat_hawker_mall_mrt_school

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,school,school_dist,num_school_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Blk 112 Jalan Bukit Merah,0.767276,6,Radin Mas Primary School,0.265618,7,HARBOURFRONT MRT STATION,0.962478,6,Radin Mas Primary School,0.265618,7
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blk 11 Telok Blangah Crescent,1.063867,5,Blangah Rise Primary School,0.372768,5,TELOK BLANGAH MRT STATION,0.260606,5,Blangah Rise Primary School,0.372768,5
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Blk 112 Jalan Bukit Merah,0.728191,6,Radin Mas Primary School,0.221545,8,HARBOURFRONT MRT STATION,0.988774,6,Radin Mas Primary School,0.221545,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,Commonwealth Crescent Market,0.299702,4,New Town Primary School,0.498291,4,COMMONWEALTH MRT STATION,0.316139,7,New Town Primary School,0.498291,4
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,Commonwealth Crescent Market,0.243151,4,New Town Primary School,0.554578,5,COMMONWEALTH MRT STATION,0.355395,7,New Town Primary School,0.554578,5


In [66]:

flat_supermarket=pd.read_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\flat_supermarket.csv")
flat_supermarket=flat_supermarket.drop(["Unnamed: 0"],axis=1)
flat_supermarket

,full_address,supermarket_dist,num_supermarket_2km
0,"45 TELOK BLANGAH DR,Singapore",0.155644,11
1,"106 BT PURMEI RD,Singapore",0.875532,16
2,"50 TELOK BLANGAH DR,Singapore",0.126816,11
3,"108 BT PURMEI RD,Singapore",0.835485,15
4,"110 SPOTTISWOODE PK RD,Singapore",0.784717,17
...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",0.129043,15
16135,"512A YISHUN ST 51,Singapore",0.686393,12
16136,"506A YISHUN AVE 4,Singapore",0.547579,13
16137,"83 COMMMONWEALTH CLOSE,Singapore",0.413755,14


In [92]:
flat_supermarket=pd.merge(flat_coord, flat_supermarket, on='full_address', how='left')
flat_supermarket

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,supermarket_dist,num_supermarket_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,0.155644,11
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,0.155644,11
2,"106 BT PURMEI RD,Singapore",1.273075,103.825657,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,0.875532,16
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,0.126816,11
4,"108 BT PURMEI RD,Singapore",1.273442,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,0.835485,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,0.102374,15
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,0.102374,15
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,0.371004,14
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,0.413755,14


In [93]:
flat_hawker_mall_mrt_school_supermarket=flat_hawker_mall_mrt_school.join(flat_supermarket[["supermarket_dist","num_supermarket_2km"]])
flat_hawker_mall_mrt_school_supermarket

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,school,school_dist,num_school_2km,supermarket_dist,num_supermarket_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5,0.155644,11
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5,0.155644,11
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Blk 112 Jalan Bukit Merah,0.767276,6,Radin Mas Primary School,0.265618,7,HARBOURFRONT MRT STATION,0.962478,6,Radin Mas Primary School,0.265618,7,0.875532,16
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blk 11 Telok Blangah Crescent,1.063867,5,Blangah Rise Primary School,0.372768,5,TELOK BLANGAH MRT STATION,0.260606,5,Blangah Rise Primary School,0.372768,5,0.126816,11
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Blk 112 Jalan Bukit Merah,0.728191,6,Radin Mas Primary School,0.221545,8,HARBOURFRONT MRT STATION,0.988774,6,Radin Mas Primary School,0.221545,8,0.835485,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6,0.102374,15
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6,0.102374,15
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,Commonwealth Crescent Market,0.299702,4,New Town Primary School,0.498291,4,COMMONWEALTH MRT STATION,0.316139,7,New Town Primary School,0.498291,4,0.371004,14
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,Commonwealth Crescent Market,0.243151,4,New Town Primary School,0.554578,5,COMMONWEALTH MRT STATION,0.355395,7,New Town Primary School,0.554578,5,0.413755,14


In [94]:
flat_park=pd.read_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\flat_park.csv")

In [100]:
flat_park=flat_park.rename(columns={"flat":"full_address"})

In [98]:
flat_park=flat_park.drop(["Unnamed: 0"],axis=1)
flat_park

,flat,park_dist,num_park_2km
0,"45 TELOK BLANGAH DR,Singapore",0.748502,7
1,"106 BT PURMEI RD,Singapore",0.862713,6
2,"50 TELOK BLANGAH DR,Singapore",0.638777,6
3,"108 BT PURMEI RD,Singapore",0.832629,6
4,"110 SPOTTISWOODE PK RD,Singapore",0.993505,8
...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",0.760222,2
16135,"512A YISHUN ST 51,Singapore",1.121209,3
16136,"506A YISHUN AVE 4,Singapore",0.968683,3
16137,"83 COMMMONWEALTH CLOSE,Singapore",0.942866,11


In [101]:
flat_park=pd.merge(flat_coord, flat_park, on='full_address', how='left')
flat_park

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,park_dist,num_park_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,0.748502,7
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,0.748502,7
2,"106 BT PURMEI RD,Singapore",1.273075,103.825657,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,0.862713,6
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,0.638777,6
4,"108 BT PURMEI RD,Singapore",1.273442,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,0.832629,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,1.847687,1
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,1.847687,1
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,0.989438,10
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,0.942866,11


In [102]:
flat_hawker_mall_mrt_school_supermarket_park=flat_hawker_mall_mrt_school_supermarket.join(flat_park[["park_dist","num_park_2km"]])
flat_hawker_mall_mrt_school_supermarket_park

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,school,school_dist,num_school_2km,supermarket_dist,num_supermarket_2km,park_dist,num_park_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5,0.155644,11,0.748502,7
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5,0.155644,11,0.748502,7
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Blk 112 Jalan Bukit Merah,0.767276,6,Radin Mas Primary School,0.265618,7,HARBOURFRONT MRT STATION,0.962478,6,Radin Mas Primary School,0.265618,7,0.875532,16,0.862713,6
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blk 11 Telok Blangah Crescent,1.063867,5,Blangah Rise Primary School,0.372768,5,TELOK BLANGAH MRT STATION,0.260606,5,Blangah Rise Primary School,0.372768,5,0.126816,11,0.638777,6
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Blk 112 Jalan Bukit Merah,0.728191,6,Radin Mas Primary School,0.221545,8,HARBOURFRONT MRT STATION,0.988774,6,Radin Mas Primary School,0.221545,8,0.835485,15,0.832629,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6,0.102374,15,1.847687,1
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6,0.102374,15,1.847687,1
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,Commonwealth Crescent Market,0.299702,4,New Town Primary School,0.498291,4,COMMONWEALTH MRT STATION,0.316139,7,New Town Primary School,0.498291,4,0.371004,14,0.989438,10
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,Commonwealth Crescent Market,0.243151,4,New Town Primary School,0.554578,5,COMMONWEALTH MRT STATION,0.355395,7,New Town Primary School,0.554578,5,0.413755,14,0.942866,11


In [103]:
flat_hawker_mall_mrt_school_supermarket_park.isna().sum()

full_address               0
Latitude                   0
Longitude                  0
month                      0
town                       0
flat_type                  0
block                      0
street_name                0
storey_range               0
floor_area_sqm             0
flat_model                 0
lease_commence_date        0
resale_price               0
remaining_lease        46739
hawker                     0
hawker_dist                0
num_hawker_2km             0
mall                       0
mall_dist                  0
num_mall_2km               0
mrt                        0
mrt_dist                   0
num_mrt_2km                0
school                     0
school_dist                0
num_school_2km             0
supermarket_dist           0
num_supermarket_2km        0
park_dist                  0
num_park_2km               0
dtype: int64

In [104]:
flat_hawker_mall_mrt_school_supermarket_park["remaining_lease"]=2021-flat_hawker_mall_mrt_school_supermarket_park["lease_commence_date"]

In [105]:
flat_hawker_mall_mrt_school_supermarket_park

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,school,school_dist,num_school_2km,supermarket_dist,num_supermarket_2km,park_dist,num_park_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,45,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5,0.155644,11,0.748502,7
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,45,Blk 11 Telok Blangah Crescent,1.157899,3,Blangah Rise Primary School,0.454797,5,TELOK BLANGAH MRT STATION,0.153912,4,Blangah Rise Primary School,0.454797,5,0.155644,11,0.748502,7
2,"106 BT PURMEI RD,Singapore",1.273075,103.825658,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,37,Blk 112 Jalan Bukit Merah,0.767276,6,Radin Mas Primary School,0.265618,7,HARBOURFRONT MRT STATION,0.962478,6,Radin Mas Primary School,0.265618,7,0.875532,16,0.862713,6
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,43,Blk 11 Telok Blangah Crescent,1.063867,5,Blangah Rise Primary School,0.372768,5,TELOK BLANGAH MRT STATION,0.260606,5,Blangah Rise Primary School,0.372768,5,0.126816,11,0.638777,6
4,"108 BT PURMEI RD,Singapore",1.273441,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,37,Blk 112 Jalan Bukit Merah,0.728191,6,Radin Mas Primary School,0.221545,8,HARBOURFRONT MRT STATION,0.988774,6,Radin Mas Primary School,0.221545,8,0.835485,15,0.832629,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,6,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6,0.102374,15,1.847687,1
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,6,Chong Pang Market & Food Centre,3.117045,0,Northoaks Primary School,0.073958,6,SEMBAWANG MRT STATION,1.040181,1,Northoaks Primary School,0.073958,6,0.102374,15,1.847687,1
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,54,Commonwealth Crescent Market,0.299702,4,New Town Primary School,0.498291,4,COMMONWEALTH MRT STATION,0.316139,7,New Town Primary School,0.498291,4,0.371004,14,0.989438,10
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,54,Commonwealth Crescent Market,0.243151,4,New Town Primary School,0.554578,5,COMMONWEALTH MRT STATION,0.355395,7,New Town Primary School,0.554578,5,0.413755,14,0.942866,11


In [106]:
flat_hawker_mall_mrt_school_supermarket_park.isna().sum()

full_address           0
Latitude               0
Longitude              0
month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
hawker                 0
hawker_dist            0
num_hawker_2km         0
mall                   0
mall_dist              0
num_mall_2km           0
mrt                    0
mrt_dist               0
num_mrt_2km            0
school                 0
school_dist            0
num_school_2km         0
supermarket_dist       0
num_supermarket_2km    0
park_dist              0
num_park_2km           0
dtype: int64

In [107]:
flat_hawker_mall_mrt_school_supermarket_park.to_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\FINAL Resale Flat Processed Data(2011-2021).csv")